# BeautyfulSoup 네이버 영화 리뷰 스크래핑

## 네이버 영화 리뷰 스크래핑

* 네이버 영화: https://movie.naver.com/movie/point/af/list.nhn

In [11]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_movie_reviews(mcode, page_num=10):

  movie_review_df = pd.DataFrame(columns=('Title', 'Score', 'Review'))
  url = 'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=' + str(mcode) + '&target=after'
  idx = 0
  for _ in range(0, page_num):
    movie_page = urllib.request.urlopen(url).read()
    movie_page_soup = BeautifulSoup(movie_page,'html.parser')

    review_list = movie_page_soup.find_all('td',{'class':'title'})

    for review in review_list:
      title = review.find('a',{'class':'movie color_b'}).get_text()
      score = review.find('em').get_text()
      review_text = review.find('a',{'class':'report'}).get('href').split(',')[2]
      movie_review_df.loc[idx] = [title, score, review_text]
      idx += 1
      print('#', end='')
    try:
      url = 'https://movie.naver.com/' + movie_page_soup.find('a',{'class':'pg_next'}).get('href')
    except:
      break

  return movie_review_df

In [15]:
movie_review_df = get_movie_reviews(18847, 1)
movie_review_df

##########

,Title,Score,Review
0,타이타닉,10,''
1,타이타닉,10,''
2,타이타닉,10,'상실을 아는 사람들이 살아가는 이야기'
3,타이타닉,10,''
4,타이타닉,10,''
5,타이타닉,10,'최고의 영화 말이 필요없음'
6,타이타닉,10,'디카프리오의 리즈시절을 볼수있어서 너무좋고요 여주도 엄청나게 예뻐서 눈이 너무 ...
7,타이타닉,10,'제대로본건 처음인데안봤으면 억울할뻔 했어요살아계신분들은 다 보셔야해요!최고의 영...
8,타이타닉,10,'내생에 가장 슬프고 아름다운 영화였다..감히 평가란걸 할수없는 영화다.'
9,타이타닉,10,'2000년 생입니다. 이 영화를 극장에서 보신 분들 정말 복받으셨네요...'


## 현재상영작 영화 리뷰 스크래핑

In [18]:
url = 'https://movie.naver.com/movie/point/af/list.nhn'
naver_movie = urllib.request.urlopen(url).read()
soup = BeautifulSoup(naver_movie, 'html.parser')
select = soup.find('select',{'id':'current_movie'})
movies = select.find_all('option')

movies_dict = {}
for movie in movies[1:]:
  movies_dict[movie.get('value')] = movie.get_text()

movie_review_df = pd.DataFrame(columns=('Title', 'Score', 'Review'))
for mcode in movies_dict:
  df = get_movie_reviews(mcode,1)
  movie_review_df = pd.concat([movie_review_df, df])

movie_review_df

########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

,Title,Score,Review
0,테넷,10,'놀란이 놀란했다. 어렵고 재밌다'
1,테넷,8,'보는내내 끊임없이 생각하고 생각하게 드는 영화'
2,테넷,10,'같은 공간에 시간을 겹쳐서 표현하는 획기적인 발상 덕분에 이해하기 쉽지 않은 영화
3,테넷,10,'재미있게 봤고 내용 다 이해감'
4,테넷,10,'이해하지 말고 느껴라 테넷'
...,...,...,...
5,왕중왕,9,'예수를 다룬 영화중에 가장 뛰어나다...'
6,왕중왕,2,'예수 같은 왕 둔 적 없다.'
7,왕중왕,10,'예수님의 왕 자리는 대통령 이나 수상이나 황제나 천왕 의 자리가 아니...
8,왕중왕,10,'20세기가 낳은 최고의 명작'
